In [31]:
import os
import yaml
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

tqdm.pandas()

In [32]:
# Carregar o CSV com tuplas únicas já processadas
csv_path = '../data/summary_calendar_unique.csv'
df = pd.read_csv(csv_path, encoding='utf-8')

# Renomear coluna TITLE para SUMMARY se necessário
if 'TITLE' in df.columns:
    df = df.rename(columns={'TITLE': 'SUMMARY'})

print(f"=== CARREGANDO TUPLAS ÚNICAS ===")
print(f"Shape: {df.shape}")
print(f"Colunas: {df.columns.tolist()}")
print(f"Primeiras 5 linhas:")
print(df.head())

# Adicionar coluna VERBALIZED se não existir
if 'VERBALIZED' not in df.columns:
    df['VERBALIZED'] = None
    print("✅ Coluna VERBALIZED adicionada")

print(f"\nTuplas ainda não verbalizadas: {df['VERBALIZED'].isnull().sum()}")
print(f"Total de tuplas: {len(df)}")

# Obter lista de calendários únicos para uso posterior
calendar_unique_list = df['CALENDAR'].unique().tolist()
print(f"Calendários únicos: {calendar_unique_list}")

=== CARREGANDO TUPLAS ÚNICAS ===
Shape: (320, 2)
Colunas: ['SUMMARY', 'CALENDAR']
Primeiras 5 linhas:
           SUMMARY     CALENDAR
0    cafe da manha  Alimentação
1  lanche da manha  Alimentação
2           almoco  Alimentação
3  lanche da tarde  Alimentação
4       pre-treino  Alimentação
✅ Coluna VERBALIZED adicionada

Tuplas ainda não verbalizadas: 320
Total de tuplas: 320
Calendários únicos: ['Alimentação', 'PUC', 'Pessoal', 'Saúde', 'Compromisso', 'Exames', 'Social', 'Games', 'Exercícios', 'Trabalho', 'Viagens']


In [33]:
yaml_path = '../verbalizer.prompt.yml'
with open(yaml_path, 'r') as file:
    verbalizer_config = yaml.safe_load(file)

# Acessar partes específicas do YAML
messages = verbalizer_config.get('messages', [])
model = verbalizer_config.get('model', '')
model_params = verbalizer_config.get('modelParameters', {})
temperature = verbalizer_config.get('modelParameters').get('temperature')
max_tokens = verbalizer_config.get('modelParameters').get('max_tokens')


print(f"Modelo: {model}")
print(f"Parâmetros do modelo: {model_params}")
print(f'Temperatura: {temperature}')
print(f'Máximo de tokens: {max_tokens}')

# Exemplo de como acessar o conteúdo das mensagens
print('Mensagens:')
print(f'System: {messages[0].get("content", "")}')
print(f'User: {messages[1].get("content", "")}')

Modelo: openai/gpt-4o
Parâmetros do modelo: {'max_tokens': 2048, 'temperature': 0.4, 'top_p': 0.8}
Temperatura: 0.4
Máximo de tokens: 2048
Mensagens:
System: Você é um assistente que recebe entradas com dois campos:
- título: string (título de uma tarefa)
- calendário: string (tipo de calendário, ex.: {{calendars}})

Sua tarefa:
1. Verifique se o campo título já está verbalizado, ou seja, se começa com um sujeito + verbo conjugado (por exemplo “Eu preciso…”, “Agendar…”, “Revisar…”).
   - Se **sim**, retorne exatamente o valor de `título`, **sem** acrescentar ou alterar nada.
2. Se **não**, gere uma frase completa em português que:
   - Comece com ”Fazer” ou outro verbo adequado ao sentido da tarefa.
   - Você pode transformar substantivos em verbos também, como "gravação" para "gravar", "comemoração" para "comemorar"
   - Preserve o sentido original do título.
   - Nunca invente ações para serem feitas. Se a ação não estiver clara na mensagem com o calendário, retorne o titulo original

In [ ]:
client = OpenAI(
    api_key=os.getenv('GEMINI_API_KEY'),
    base_url=os.getenv('GEMINI_BASE_URL')
)

def verbalize_task(row):
    """
    Verbaliza uma tarefa com base no resumo e no calendário fornecidos.
    """
    try:
        response = client.chat.completions.create(
            model="gemini-2.5-flash",
            temperature=verbalizer_config.get('modelParameters').get('temperature'),
            max_tokens=verbalizer_config.get('modelParameters').get('max_tokens'),
            messages=[
                {
                    "role": "system",
                    "content": messages[0].get('content').replace("{{calendars}}", str(calendar_unique_list))
                },
                {
                    "role": "user",
                    "content": messages[1].get('content').replace("{{summary}}", row['SUMMARY']).replace("{{calendar}}", row['CALENDAR'])
                }
            ],
        )

        row['VERBALIZED'] = response.choices[0].message.content
        return row
    except Exception as e:
        row['VERBALIZED'] = None
        return row

df = df.progress_apply(verbalize_task, axis=1)


  1%|          | 2/320 [00:01<03:30,  1.51it/s]

Resultado verbalizado: ```json
{
  "task": "Tomar café da manhã",
  "type": "generated"
}
```


  1%|          | 3/320 [00:02<05:34,  1.06s/it]

Resultado verbalizado: ```json
{
  "task": "Lanchar de manhã",
  "type": "generated"
}
```


  1%|▏         | 4/320 [00:04<06:20,  1.21s/it]

...

Resultado verbalizado: ```json
{
  "task": "Viajar para Petrópolis",
  "type": "generated"
}
```


100%|██████████| 320/320 [11:17<00:00,  1.83s/it]

Resultado verbalizado: ```json
{
  "task": "Visitar Petrópolis e as torres",
  "type": "generated"
}
```


100%|██████████| 320/320 [11:18<00:00,  2.12s/it]

Resultado verbalizado: ```json
{
  "task": "Viajar para Petrópolis com Juliana",
  "type": "generated"
}
```


# Exportar o DataFrame para um arquivo CSV


In [35]:
df.to_csv('../data/verbalized_tasks.csv', index=False, encoding='utf-8')